# Prerequesites

In [ ]:
import extraTypes._

# Sinus

First we override the number input type for `Int` (default to `<input type="number">`) with an input type `range` using the `SliderType` from `5` to `15` by `2`.

In [ ]:
implicit val ITI:types.InputType[Int] = SliderType[Int](5, 15, 2)
val si = new InputBox(5, "slider")

Then we define the function to create a sine curve using a scaling factor `k`: 

$\sin(k*x)$

In [ ]:
def sin(k:Int) = (-math.Pi to math.Pi by 0.1).toList map (x => math.sin(k*x))

Let's plot that in a regular `LineChart`

In [ ]:
val plot = widgets.LineChart(sin(1), fields=Some(("_1", "_2")), maxPoints=100)

Now the **reactive** part, in order to adapt `k` we will use the slider and see the effect on the sine wave.

To do this, we ask the **events** in the slider (`si`) to be published into a dedicated **`Observer`** that will create a new sine wave using `k` and then ask the plot to update using `plot.applyOn` with the new data.

In [ ]:
si.currentData --> Connection.fromObserver{k: Int =>
                                            val s = sin(k) 
                                            plot.applyOn(s)
                                          }

# Beta

We can also show what's the density of the $\beta$ distribution using its two parameters:
* $\alpha$ ~ the count of successes
* $\beta$ ~ the count of failures

In [ ]:
implicit val ITI:types.InputType[Int] = SliderType[Int](1, 1000, 1)
val alpha = new InputBox(0, "alpha")
val beta = new InputBox(0, "beta")
alpha ++ beta

Now, we can create the $\beta$ density function to generate the plot.

In [ ]:
def e(a:Int, b:Int) = {
  val d = com.cra.figaro.library.atomic.continuous.Beta(a, b)
  val s = (0d to 1d by 0.01).toList
  s map (x => (x, d.density(x)))
}
var a = 1
var b = 1
val betaPlot = widgets.LineChart(e(1, 1), fields=Some(("_1", "_2")))

And finally, we simply update the value depending if we update $\alpha$ or $\beta$ (note that we could have merge the streams...)

In [ ]:
alpha.currentData --> Connection.fromObserver{ i: Int =>
                                          a = i
                                          val s = e(a, b) 
                                          betaPlot.applyOn(s)
                                       }
beta.currentData --> Connection.fromObserver{ i: Int =>
                                          b = i
                                          val s = e(a, b) 
                                          betaPlot.applyOn(s)
                                       }